# Carga de archivos

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 48.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=e3c6d914800be65bd82acd6c6253bf814e87355188237827fe1a9695a16d2203
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
!apt install openjdk-8-jdk-headless -qq

The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155629 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Selecting previously unselected package openjdk-8-

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"14sjAJe8wBi7N_1xxJFCm7Mtzrm_gFiPj"})   # replace the id with id of file you want to access
downloaded.GetContentFile('business.csv') 

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

Aca cargo solamente la columna 'hours' del df, y saco los na

In [ ]:
sqlContext = SQLContext(sc)
dfBusiness = (spark.read\
      .option("multiline", "true")\
      .option("quote", '"')\
      .option("header", "true")\
      .option("escape", "\\")\
      .option("escape", '"')\
      .csv('business.csv'))\
      .select(['hours'])\
      .dropna()
rdd = dfBusiness.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


# Calculo

In [ ]:
negociosTotales = rdd.count()
negociosTotales

127123

In [ ]:
rdd.first()

Row(hours="{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'}")

Hago una funcion para ver si tal hora esta en un rango horario (si cierra 21hs lo considero cerrado)

In [ ]:
def estaEnRango(string, hora):
  l = string.split("-")
  min = float(l[0].split(":")[0]) + ( float(l[0].split(':')[1]) / 60) 
  max = float(l[1].split(":")[0]) + ( float(l[1].split(':')[1]) / 60)
  return True if (hora-min) * (hora-max) < 0 else False

1-Veo que el formato del csv es como el de un dict de python, importo la libreria ast para hacerlo 

2- filtro para obtener los que estan abiertos los domingos y a las 21hs

In [ ]:
import ast
rddFiltered = rdd.map(lambda x: ast.literal_eval(x[0]))\
                 .filter(lambda x: 'Sunday' in x.keys() and estaEnRango(x["Sunday"], 21))

In [ ]:
negociosCumplen = rddFiltered.count()
negociosCumplen

15525

In [ ]:
# Respuesta: 
negociosCumplen / negociosTotales

0.12212581515540068